# Introduction

Needless to say, run all the way!

In [0]:
#@title Cek GPU
# Check the device, is it K80 or T4, if it is K80 ...!
! /opt/bin/nvidia-smi

# The first step is to prepare the workspace
At this step you can choose two ways
1. Using the default workspace, you don't need to upload it yourself, only for familiar operation.
2. Upload your workspace to the specified directory via Google Drive.  

In [0]:
#@title Start mounting 
#Mount Google Cloud Disk
#Click the link to authorize, copy the authorization code, fill in the box, and then press Enter。

from google.colab import drive
drive.mount('/content/drive', force_remount=True)

#Create DeepFaceLab directory, and enter the directory
%cd /content/drive/My Drive/
!mkdir DeepFaceLab
%cd /content/drive/My Drive/DeepFaceLab/


There are two options for this link:
If you are using it for the first time, you can load the sample project and run it backwards.
If you run it twice, you don't need to load it.
If you want to upload local material, you can use the pack script to pack the material into faceset.pak to the aligned directory. There is no need to decompress it. You can directly train and convert. The loading speed will be fast.




In [0]:
#@title Load the sample project workspace 
# 1. Practice can use this line directly git clone一个workspace
%cd /content/drive/My Drive/DeepFaceLab/
!rm -rf workspace
!git clone https://github.com/dream80/DFLWorkspace.git workspace

#Second step install DeepFaceLab
Get the source code, install dependencies, and choose the version based on your situation

1229: The last release of 2019

last: latest version.  

In [0]:
#@title start installation

Version = "v1.12.29" #@param ["v1.12.29", "last"]
%cd /content/drive/My Drive/DeepFaceLab
!rm -fr DeepFaceLab_Colab
if Version=="620":
  print("620 Loading....")
  # Get DFL source code v1.6.1 stable version
  !git clone -b v1.6.1 https://github.com/dream80/DeepFaceLab_Colab.git
  %cd /content/drive/My Drive/DeepFaceLab/DeepFaceLab_Colab
  !pip install -r requirements-colab.txt  
elif Version=="last":
  print("Loading the latest version....")
  !git clone https://github.com/dream80/DeepFaceLab_Colab.git
  %cd /content/drive/My Drive/DeepFaceLab/DeepFaceLab_Colab
  !pip install -r requirements-colab.txt
else:
  print(Version+"Loading....")
  cmd="clone -b "+Version+" https://github.com/dream80/DeepFaceLab_Colab.git"  
  !git $cmd
  %cd /content/drive/My Drive/DeepFaceLab/DeepFaceLab_Colab
  !pip install -r requirements-colab.txt

!pip install --upgrade scikit-image
!sudo apt-get install cuda-10-0


# Step 3. Extract faces

This step is actually divided into two. I chose src for the first time and dst for the second time. Many people who don't know Deepfacelab may only order one, and I will get an error later.
  
In this step, there are actually three small steps, decomposing the video, extracting faces, and packing material. Split them if needed. This is generally not necessary. 





In [0]:
#@title Start extraction
target = "src" #@param ["src","dst"]
%cd /content/drive/My Drive/DeepFaceLab/DeepFaceLab_Colab
if target=="src":
  !python main.py videoed extract-video --input-file ../workspace/data_src.mp4 --output-dir ../workspace/data_src/
  !python main.py extract --input-dir ../workspace/data_src --output-dir ../workspace/data_src/aligned --detector s3fd --debug-dir ../workspace/data_src/aligned_debug
  !python main.py util --input-dir ../workspace/data_src/aligned  --pack-faceset
else:
  !python main.py videoed extract-video --input-file ../workspace/data_dst.mp4 --output-dir ../workspace/data_dst/
  !python main.py extract --input-dir ../workspace/data_dst --output-dir ../workspace/data_dst/aligned --detector s3fd --debug-dir ../workspace/data_dst/aligned_debug
  !python main.py util --input-dir ../workspace/data_dst/aligned  --pack-faceset


In [0]:
#@title Start sorting
target = "src" #@param ["src","dst"]
order = "hist" #@param ["blur", "face", "face-dissim", "face-yaw", "face-pitch", "hist", "hist-dissim", "brightness", "hue", "black", "origname", "oneface", "final", "final-no-blur"]

if target=="src":
  #SrcSort, you can view the results through Google Cloud Disk, delete the bad pictures
  cmd = "main.py sort --input-dir ../workspace/data_src/aligned --by "+order
  
else:
  cmd = "main.py sort --input-dir ../workspace/data_dst/aligned --by "+order

!python $cmd

# Step 4. Train the model


* Supports SAEHD, Quick96, H128, SAE, DF, LIAEF128 and other models, choose the model according to your own situation.
* You need to configure the model parameters for the first time. If you don't understand, please press Enter to return to the default.
* If you don't want to train, you can click Stop. An exception will be thrown when you stop, but it doesn't matter. Can continue training next time



In [0]:
#@title Start training
Model = "SAEHD" #@param ["SAEHD", "SAE", "H128", "LIAEF128", "DF", "AVATAR","Quick96"]

%cd "/content/drive/My Drive/DeepFaceLab/DeepFaceLab_Colab/"
cmd = "main.py train --training-data-src-dir ../workspace/data_src/aligned --training-data-dst-dir ../workspace/data_dst/aligned --model-dir ../workspace/model --model "+Model+" --no-preview"
!python $cmd

** For model preview method, please refer to: **
https://www.wxp123.me/p/329

Brief description: left-> file-> / driver /..../ model / * preview * .jpg-> double-click
  
This method is more convenient than previewing in the script and previewing on the network disk, and does not need to start the preview configuration!

# Step 5. Convert the output

What are the candidates for training the model?
For example training H128, here we choose H128


In [0]:
#@title Start converting pictures

Model = "SAEHD" #@param ["SAEHD", "SAE", "H128", "LIAEF128", "DF", "AVATAR", "Quick96"]
%cd "/content/drive/My Drive/DeepFaceLab/DeepFaceLab_Colab/"
cmd = " main.py convert --input-dir ../workspace/data_dst --output-dir ../workspace/data_dst/merged --aligned-dir ../workspace/data_dst/aligned --model-dir ../workspace/model --model " + Model
!python $cmd

In [0]:
#@title Start picture to video
!python main.py videoed video-from-sequence --input-dir ../workspace/data_dst/merged --output-file ../workspace/result.mp4 --reference-file ../workspace/data_dst.mp4

#Step 6. Continue training
You don't need to perform all the above steps when you start training for the second time or continue training after you are offline. Just a few simple steps below.
1. Mount cloud disk
2. Installation dependencies
3. Start training


In [0]:
#@title One-click operation
#Mounted Google Cloud Disk
#Click the link to authorize, copy the authorization code, fill in the box, and then press Enter.

Model = "SAEHD" #@param ["SAEHD", "SAE", "H128", "LIAEF128", "DF", "AVATAR" , "Quick96"]


from google.colab import drive
drive.mount('/content/drive', force_remount=True)


# Enter DeepFaceLab_Colab directory
%cd /content/drive/My Drive/DeepFaceLab/DeepFaceLab_Colab

# Install Python dependencies
!pip install -r requirements-colab.txt
!pip install --upgrade scikit-image
!sudo apt-get install cuda-10-0

# Start training SAE, if it is another model, modify the following parameters, such as H128。
cmd ="main.py train --training-data-src-dir ../workspace/data_src/aligned --training-data-dst-dir ../workspace/data_dst/aligned --model-dir ../workspace/model --model " + Model +" --no-preview"
!python $cmd 

# Tools

Package footage. Can speed up uploading, downloading, and loading.

In [0]:
#@title Material packaging

target = "src" #@param ["src","dst"]
%cd /content/drive/My Drive/DeepFaceLab/DeepFaceLab_Colab
if target=="src":
  !python main.py util --input-dir ../workspace/data_src/aligned  --pack-faceset
else:
 !python main.py util --input-dir ../workspace/data_dst/aligned  --pack-faceset
 
  

In [0]:
#@title Material unpacking

target = "src" #@param ["src","dst"]
%cd /content/drive/My Drive/DeepFaceLab/DeepFaceLab_Colab
if target=="src":
  !python main.py util --input-dir ../workspace/data_src/aligned --unpack-faceset
else:
 !python main.py util --input-dir ../workspace/data_dst/aligned --unpack-faceset
 
  

In [0]:
#@title Material enhancement

target = "src" #@param ["src"]
%cd /content/drive/My Drive/DeepFaceLab/DeepFaceLab_Colab
if target=="src":
  !python main.py facesettool enhance --input-dir ../workspace/data_src/aligned
else:
  !python main.py facesettool enhance --input-dir ../workspace/data_dst/aligned
 
  